In [1]:
import sys
sys.path.append('../build')   # assume an out-of-source build

import numpy as np
import exafmm as fmm
import exafmm.laplace as laplace

In [2]:
laplace.__doc__

"exafmm's submodule for Laplace kernel"

### Create sources and targets

In [3]:
nsrcs = 100000
ntrgs = 100000

# generate random positions for particles
src_coords = np.random.random((nsrcs, 3))
trg_coords = np.random.random((nsrcs, 3))

# generate random charges for sources
src_charges = np.random.random(nsrcs)

In [4]:
# create a list of source instances
sources = laplace.init_sources(src_coords, src_charges)

# create a list of target instances
targets = laplace.init_targets(trg_coords)

### Create an LaplaceFMM instance

In [5]:
fmm = laplace.LaplaceFMM()
fmm.p = 10
fmm.nsurf = 6*(fmm.p-1)*(fmm.p-1) + 2
fmm.ncrit = 400
fmm.depth = 4

### Setup FMM

Given sources, targets and FMM configurations, `setup()` builds the tree and interaction list and pre-compute invariant matrices.

Set `skip_P2P` to `True` to skip near-field interactions for BEM applications.

In [6]:
skip_P2P = False
tree = laplace.setup(sources, targets, fmm, skip_P2P)

### Evaluate

`evaluate()` triggers the evaluation and returns a $n_{trg} \times 4$ `numpy.ndarray`.
The $i$-th row of `trg_values` starts with the potential value of the $i$-th target, followed by its three gradient values.

In [7]:
trg_values = laplace.evaluate(tree, fmm)

P2M                  : 2.0872800e-01
M2M                  : 1.0536300e-01
P2L                  : 7.2000000e-05
M2P                  : 5.6000000e-05
P2P                  : 4.2103000e-02
M2L                  : 7.4878000e-01
L2L                  : 9.9013000e-02
L2P                  : 1.4872900e-01


In [8]:
trg_values[6]

array([ 7695.63176065,  3473.07102295, -1430.86648635,  5882.21385108])

### Check Accuracy

`fmm.verify(tree.leafs)` returns L2-norm the relative error of potential and gradient in a list. It only works when `skip_P2P` is set to `False`.

In [9]:
fmm.verify(tree.leafs)

[6.928890228592761e-10, 1.3233326176140564e-07]

### Update charges of sources and run FMM iteratively

In [10]:
niters = 10

for i in range(niters):
    print('-'*10 + ' iteration {} '.format(i) + '-'*10)  # print divider between iterations
    
    src_charges = np.random.random(nsrcs)          # generate new random charges
    laplace.update_charges(tree, src_charges)      # update charges
    laplace.clear_values(tree)                     # clear values
    trg_values = laplace.evaluate(tree, fmm)       # evaluate potentials

    print("Error: ", fmm.verify(tree.leafs))       # check accuracy

---------- iteration 0 ----------
P2M                  : 2.0134800e-01
M2M                  : 1.0770800e-01
P2L                  : 1.6200000e-03
M2P                  : 5.6000000e-05
P2P                  : 4.2351000e-02
M2L                  : 7.3041700e-01
L2L                  : 9.8867000e-02
L2P                  : 1.4920200e-01
Error:  [7.466645128068746e-10, 1.4655429658435553e-07]
---------- iteration 1 ----------
P2M                  : 1.4060100e-01
M2M                  : 9.9754000e-02
P2L                  : 7.1000000e-05
M2P                  : 5.4000000e-05
P2P                  : 4.0229000e-02
M2L                  : 7.7383300e-01
L2L                  : 1.0025900e-01
L2P                  : 1.7057300e-01
Error:  [6.584910011033258e-10, 1.2410547665183176e-07]
---------- iteration 2 ----------
P2M                  : 1.3778000e-01
M2M                  : 1.0375200e-01
P2L                  : 7.0000000e-05
M2P                  : 5.1000000e-05
P2P                  : 4.0150000e-02
M2L      